# Métodos de punto interior
Los métodos de punto interior permiten resolver problemas de la forma 


\begin{align*}
\min &f_0(x)\\
\text{s.t.} &f_i(x) \ge 0, i = 1,\dots,m\\
&Ax=b
\end{align*}

Donde $f_0, \dots, f_m:\mathbb R^n \rightarrow \mathbb R$ son convexas y de clase $\mathcal C^2$, y $A\in \mathbb R^{p\times n}$ con $\text{rango}(A)=p<n$

Aplicando el método de newton a una secuencia de problemas con restricciones de igualdad o a una secuencia de versiones modificadas de las condiciones KKT.

In [29]:
# imports
import numpy as np
import sympy as sp
from IPython.display import display_latex, Latex


**Ejemplo** 

Implementar el método de la barrera, y primal-dual para resolver el problema cuadrático sin restricciones de igualdad


\begin{align*}
\min &\frac{1}{2} x^TPx + q^T x\\
\text{s.t.} &Ax \le b
\end{align*}

Con $A\in \mathbb R^{m\times n}$

In [30]:
n = 2
m = 6
A = np.random.rand(m, n)
b = np.random.rand(m)

P = np.diag(np.random.random(n))
q = np.random.rand(n)
x = np.random.rand(n)


In [31]:
x1, x2, x3 = sp.symbols("x1, x2, x3")
x_sym = sp.Matrix([x1, x2, x3])

display_latex(Latex(f"$\\text{{min}}\\frac{{1}}{{2}}x^T{sp.latex(sp.N(sp.Matrix(P), 5))}x + {sp.latex(sp.N(sp.Matrix(q), 5))}x$"))
display_latex(Latex(f"$\\text{{s.t. }}{sp.latex(sp.N(sp.Matrix(A), 5))}{sp.latex(x_sym)} \le {sp.latex(sp.N(sp.Matrix(b), 5))}$"))


### Método de la barrera

Se basa en resolver una secuencia de problemas de minimización sin restricciones (o con restricciones lineales), utilizando el último punto encontrado como punto de partida para el siguiente problema de minimización sin restricciones.

#### Algoritmo 1: resolviendo un sub-problema 

In [46]:
from scipy.optimize import minimize


def barrier_method_original(f, g, x, t=2, nu=2, eps=1e-6, MAX_ITER=100):
    phi = lambda x: -sum([np.log(-gi(x)) for gi in g])
    m = len(g)
    sub_problem = lambda x, t: t * f(x) + phi(x)
    for k in range(1, MAX_ITER):
        x = minimize(sub_problem, x, t).x
        error = m / t
        print(f"{k:02d}, {x}, {error:0.8f}")
        if abs(error) < eps:
            return x
        t = nu * t


In [47]:
f = lambda x: 0.5 * np.dot(x, P @ x) + np.dot(q, x)
g = [lambda x: np.dot(A[i], x) - b[i] for i in range(len(A))]
x0 = np.zeros(n)
print("iter\tbrecha de dualidad")
x_opt = barrier_method_original(f, g, x0)
print(f"x_opt = {x_opt}")


iter	brecha de dualidad
01, [-1.73037215 -2.03170212], 3.00000000
02, [-1.39767503 -1.57631755], 1.50000000
03, [-1.18024345 -1.27870456], 0.75000000
04, [-1.04486652 -1.09340527], 0.37500000
05, [-0.96528782 -0.98448068], 0.18750000
06, [-0.92101799 -0.92388564], 0.09375000
07, [-0.89742752 -0.89159583], 0.04687500
08, [-0.8852083  -0.87487056], 0.02343750
09, [-0.87898344 -0.86635015], 0.01171875
10, [-0.87584088 -0.86204875], 0.00585938
11, [-0.87426195 -0.85988754], 0.00292969
12, [-0.8734705  -0.85880425], 0.00146484
13, [-0.8730743  -0.85826193], 0.00073242
14, [-0.87287605 -0.8579906 ], 0.00036621
15, [-0.87277692 -0.8578549 ], 0.00018311
16, [-0.87272735 -0.85778704], 0.00009155
17, [-0.87270253 -0.85775312], 0.00004578
18, [-0.87269016 -0.85773614], 0.00002289
19, [-0.87268395 -0.85772765], 0.00001144
20, [-0.87268085 -0.85772341], 0.00000572
21, [-0.8726793  -0.85772128], 0.00000286
22, [-0.87267853 -0.85772022], 0.00000143
23, [-0.87267815 -0.85771972], 0.00000072
x_opt = [-

#### Algoritmo 2: resolviendo un sistema lineal de ecuaciones

En el método de barrera, el paso de Newton $\Delta x_{n t}$, y la variable dual asociada están dadas por las ecuaciones lineales
$$
\left[\begin{array}{cc}
t \nabla^2 f_0(x)+\nabla^2 \phi(x) & A^T \\
A & 0
\end{array}\right]\left[\begin{array}{c}
\Delta x_{\mathrm{nt}} \\
\nu_{\mathrm{nt}}
\end{array}\right]=-\left[\begin{array}{c}
t \nabla f_0(x)+\nabla \phi(x) \\
0
\end{array}\right]
$$
En esta sección mostramos cómo estos pasos de Newton para el problema de centrado pueden interpretarse como pasos de Newton para resolver directamente las ecuaciones KKT modificadas
$$
\begin{aligned}
\nabla f_0(x)+\sum_{i=1}^m \lambda_i \nabla f_i(x)+A^T \nu & =0 \\
-\lambda_i f_i(x) & =1 / t, \quad i=1, \ldots, m \\
A x & =b
\end{aligned}
$$

In [34]:
def barrier_method(P, q, A, b, m, n, TOL=1e-3, NTTOL=1e-6):
    """
    A in R^mxn
    """
    MAXITERS = 200
    ALPHA = 0.01
    BETA = 0.5
    MU = 20
    x = np.zeros(n)
    t = 1
    for iter in range(MAXITERS):
        y = -(A @ x - b)
        val = t * (0.5 * np.dot(x, P @ x) + np.dot(q, x)) - np.sum(np.log(y))
        grad = t * (P @ x + q) + A.T @ (1 / y)
        hess = t * P + A.T @ np.diag(1 / y ** 2) @ A
        v = np.linalg.solve(-hess, grad)
        fprime = np.dot(grad, v)
        s = 1
        dy = -A @ v
        while min(y + s * dy) <= 0:
            s = BETA * s
        while (t * (0.5 * np.dot(x + s * v, P @ (x + s * v)) + np.dot(q, x + s * v)) - np.sum(np.log(y + s * dy)) >= val + ALPHA * s * fprime):
            s = BETA * s
        x = x + s * v
        if -fprime < NTTOL:
            gap = m / t
            print(f"{iter:02d}\t{gap}")
            if gap < TOL:
                break
            t = MU * t
    return x


In [35]:
print("iter\tbrecha de dualidad")
x_opt = barrier_method(P, q, A, b, m, n)
print(f"x_opt = {x_opt}")


iter	brecha de dualidad
06	6.0
09	0.3
12	0.015
14	0.00075
x_opt = [-0.87323781 -0.85805794]


### Método primal-dual

Los métodos primarios de doble punto interior son muy similares al método de barrera, con algunas diferencias.


$$
\left[\begin{array}{cc}
H & A^T \\
A & 0
\end{array}\right]\left[\begin{array}{c}
\Delta x_{\mathrm{nt}} \\
\nu_{\mathrm{nt}}
\end{array}\right]=-\left[\begin{array}{l}
g \\
0
\end{array}\right]
$$
donde
\begin{align*}
H&=t \nabla^2 f_0(x)+\sum_{i=1}^m \frac{1}{f_i(x)^2} \nabla f_i(x) \nabla f_i(x)^T+\sum_{i=1}^m \frac{1}{-f_i(x)} \nabla^2 f_i(x)\\
g&= t\nabla f_0(x)+\sum_{i=1}^m\frac{1}{-f_i(x)}\nabla f_i(x)
\end{align*}

In [36]:
def primal_dual_method(P, q, A, b, m, n, TOL=1e-3, NTTOL=1e-6):
    MAXITERS = 200
    TOL = 1e-6
    RESTOL = 1e-8
    MU = 10
    ALPHA = 0.01
    BETA = 0.5
    x = np.zeros(n)
    s = b - A @ x
    z = 1 / s
    for iters in range(MAXITERS):
        gap = np.dot(s, z)
        print(f"{iters:02d}\t{gap}")
        res = P @ x + q + A.T @ z
        if gap < TOL and np.linalg.norm(res) < RESTOL:
            break
        tinv = gap / (m * MU)
        sol = -np.linalg.solve(np.r_[(np.c_[P, A.T], np.c_[A, np.diag(-s / z)])],
                               np.r_[P @ x + q + A.T @ z,
                                     -s + tinv * (1 / z)])
        dx = sol[:n]
        dz = sol[n: n + m]
        ds = -A @ dx

        r = np.r_[P @ x + q + A.T @ z,
                  z * s - tinv]

        step = min(1.0, 0.99 / max(-dz / z))
        while (np.min(s + step * ds) <= 0):
            step = BETA * step

        newz = z + step * dz
        newx = x + step * dx
        news = s + step * ds
        newr = np.r_[P @ newx + q + A.T @ newz,
                     newz * news - tinv]
        while (np.linalg.norm(newr) > (1 - ALPHA * step) * np.linalg.norm(r)):
            step = BETA * step
            newz = z + step * dz
            newx = x + step * dx
            news = s + step * ds
            newr = np.r_[P @ newx + q + A.T @ newz,
                         newz * news - tinv]

        x = x + step * dx
        z = z + step * dz
        s = b - A @ x
    return x


In [37]:
print("iter\tbrecha de dualidad")
x_opt = primal_dual_method(P, q, A, b, m, n)
print(f"x_opt = {x_opt}")


iter	brecha de dualidad
00	6.0
01	0.6320226693748787
02	0.17301415088653915
03	0.08419472685667566
04	0.011422752111251986
05	0.0015228008409967602
06	9.287008649583438e-05
07	9.291292353856054e-06
08	9.29165810267434e-07
x_opt = [-0.87267845 -0.85771958]
